In [1]:
import numpy as np
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img
from skimage.transform import resize
import matplotlib.pyplot as plt

In [2]:
# Завантаження даних
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Зміна розміру зображень до 32x32 пікселів
x_train_resized = np.array([resize(img, (32, 32)) for img in x_train])
x_test_resized = np.array([resize(img, (32, 32)) for img in x_test])

# Перетворення даних
x_train_resized = np.repeat(np.expand_dims(x_train_resized, -1), 3, -1)
x_test_resized = np.repeat(np.expand_dims(x_test_resized, -1), 3, -1)
x_train_resized = np.asarray(x_train_resized).astype('float32') / 255
x_test_resized = np.asarray(x_test_resized).astype('float32') / 255
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [3]:
# Завантаження VGG16
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Замороження згорткових шарів VGG16
vgg_base.trainable = False

In [4]:
# Створення моделі
model = Sequential()
model.add(vgg_base)
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [5]:
# Компіляція моделі
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [6]:
# Навчання моделі
history = model.fit(x_train_resized, y_train, epochs=5, batch_size=64, validation_data=(x_test_resized, y_test))

# Оцінка моделі на тестових даних
test_loss, test_acc = model.evaluate(x_test_resized, y_test)
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)

Epoch 1/5
938/938 [==============================] - 1198s 1s/step - loss: 2.3083 - accuracy: 0.1105 - val_loss: 2.2914 - val_accuracy: 0.1012
Epoch 2/5
938/938 [==============================] - 1213s 1s/step - loss: 2.2617 - accuracy: 0.1594 - val_loss: 2.2042 - val_accuracy: 0.2343
Epoch 3/5
938/938 [==============================] - 1209s 1s/step - loss: 2.1241 - accuracy: 0.2245 - val_loss: 2.0305 - val_accuracy: 0.2615
Epoch 4/5
938/938 [==============================] - 1245s 1s/step - loss: 1.9578 - accuracy: 0.2756 - val_loss: 1.9295 - val_accuracy: 0.2768
Epoch 5/5
236/313 [=====================>........] - ETA: 51s - loss: 1.7960 - accuracy: 0.3108

In [14]:
# Візуалізація точності на тренувальному та валідаційному наборах даних
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()

# Візуалізація втрат на тренувальному та валідаційному наборах даних
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.show()

NameError: NameError: name 'history' is not defined

In [ ]:
'''
Висновки: Використання згорткової основи VGG16 може бути ефективним способом покращити точність моделі для завдань класифікації зображень. 
Однак це також може вимагати більших обчислювальних ресурсів. 

'''